In [ ]:
from astropy.io import ascii, fits
from astropy.table import Table, vstack
import numpy as np
import matplotlib.pyplot as plt
import es_gen
import pdb
import glob
from scipy.interpolate import interp1d
from astropy.convolution import Gaussian1DKernel, convolve
import os
import es_gen
from sp_engine import subaruSpec, measuredArray, stellModel
from specutils.io import read_fits


%matplotlib inline

## Originally, don't think H-alpha was covered

In [ ]:
s = subaruSpec('kic1255_fits/w111345.fits')

In [ ]:
s.defaultNormRegion

In [ ]:
plt.plot(s.wave,s.normFlux)
plt.ylim(0,2)
plt.xlim(6560,6590)
plt.axvline(6563.,color='red')

## In a re-analysis, Kento did recover the H-alpha
Collect all red spectra and combine

In [ ]:
fList = glob.glob('new_k1255_fits_w_halpha/w*.fits')
redCCDList = []
for oneFile in fList:
    s = subaruSpec(oneFile)
    if s.ccdSide == 'red':
        redCCDList.append(oneFile)

In [ ]:
redCCDList

In [ ]:
nFile = len(redCCDList)
timeArray = []
for ind,oneFile in enumerate(redCCDList):
    baseName = os.path.basename(oneFile)
    s = subaruSpec(oneFile)
    path1 = '../hirano/kic1255_20150827/{}'.format(baseName)
    path2 = '../hirano/kic1255_20150828/{}'.format(baseName)
    if os.path.exists(path1):
        head = fits.getheader(path1)
    elif os.path.exists(path2):
        head = fits.getheader(path2)
    else:
        print('No original header found for {}'.format(basename))
        head = {'DATE-OBS':"NAN",'UT-STR':"NAN"}
    timeArray.append("{}T{}".format(head['DATE-OBS'],head['UT-STR']))
    
    if ind == 0:
        ## Choose the wavelength grid the first time through
        pts = (s.wave > 6559.2) & (s.wave < 6570.)
        waveGrid = s.wave[pts]
        allFlux = np.zeros([len(waveGrid),nFile])
        ptsForNormalization = (waveGrid > 6558.) & (waveGrid < 6560.)
        
    f = interp1d(s.wave,s.normFlux)
    interpolatedFlux = f(waveGrid)
    allFlux[:,ind] = interpolatedFlux/np.median(interpolatedFlux[ptsForNormalization])
    
    #print("File {}, Median={}".format(oneFile,np.median(allFlux[:,ind])))

In [ ]:
medSpec = np.nanmedian(allFlux,axis=1)
medSpecObj = measuredArray(waveGrid,medSpec,Res=6e4)
medSpecObj.removePoly()
## Get Errors from the standard deviation
stdSpec = np.nanstd(allFlux,axis=1)
medSNR = np.median(medSpec/stdSpec)
medSNR

In [ ]:
plt.plot(waveGrid,medSpec)

In [ ]:
allFlux.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
for ind,oneRow in enumerate(allFlux.transpose()):
    ax.plot(waveGrid,oneRow - 0.1 * ind,label=timeArray[ind][0:16])
ax.set_xlabel("Wavelength ($\AA$)")
ax.set_ylabel("Normalized Flux")
ax.legend()
ax.set_xlim(6557,6566)
fig.savefig("plots/h_alpha_subaru.pdf")